## Pull Freesurfer Subcortical Volumes
This notebook is designed to extract and visualize freesurfer volume data

In [3]:
from pandas import DataFrame, Series, read_table
import datetime

# Set study variables
fs_subjs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
output_dir = '/home/camachocm2/Analysis/aggregate_anats/results'
subjects_list = output_dir + '/included_subs.txt'
subjects = open(subjects_list).read().splitlines()

# Determine what regions from the aseg.stats files to aggregate
regions_of_interest = ['Left-Caudate','Left-Putamen','Left-Pallidum','Left-Accumbens-area',
                       'Left-Thalamus-Proper','Right-Caudate','Right-Putamen','Right-Pallidum',
                       'Right-Accumbens-area','Right-Thalamus-Proper','Right-Hippocampus',
                       'Left-Hippocampus','Right-Amygdala','Left-Amygdala']

# Set up dataframe to store data
data = DataFrame(columns=['SubjectID','eTIV'] + regions_of_interest)

In [4]:
# Loop through all subjects
for subidx in range(0,len(subjects)):
    sub = subjects[subidx]
    data.loc[subidx,'SubjectID']=sub
    sub_data = read_table(fs_subjs_dir + '/' + sub + '/stats/aseg.stats', header=None, delim_whitespace=True, 
                          index_col = None, comment='#')
    
    # Extract subcortical volumes
    for label in regions_of_interest:
        for a in range(0,len(sub_data[4])):
            if label==sub_data[4][a]:
                data.loc[subidx,label] = sub_data.iloc[a,3]
    
    # Extract estimated total intracranial volume or eTIV
    all_lines = open(fs_subjs_dir + '/' + sub + '/stats/aseg.stats').read().splitlines()
    for line in all_lines:
        if "Estimated Total Intracranial Volume" in line:
            eTIV_line = line
            break
    eTIV_line = eTIV_line.split(',')
    data.loc[subidx,'eTIV'] = eTIV_line[3]
    
# save the compiled data with today's date
data_file = output_dir + '/aseg_data_{date:%Y%m%d-%H%M%S}.csv'.format(date=datetime.datetime.now())
data.to_csv(data_file)